In [1]:
import os, sys
sys.path.append( os.path.join('..','..','src','mlspec_flow') )

from utils.paths import saved_model_dir

In [2]:
flow_dir   = "TaxiTS2017MinimalV01"
model_name = "20210521V02"
numeric_columns     = ['pickup_latitude',
                       'pickup_longitude',
                       'dropoff_latitude',
                       'dropoff_longitude']
categorical_columns = []
label="trip_seconds"
os.environ["MODEL_PATH"] = saved_model_dir(flow_dir,model_name)

In [3]:
os.environ["MODEL_PATH"]

'gs://mlteam-ml-specialization-2021-taxi/tft_flows/TaxiTS2017MinimalV01/models/20210521V02/1/export/1621856445/'

In [4]:
import json

json_path = "req.json"
instance=dict(list(zip(numeric_columns,[0 for _ in numeric_columns]))+list(zip(categorical_columns, ["" for _ in categorical_columns])))
if "trip_start" in instance:
    instance["trip_start"] = "2017-03-03 Fry-09:00"
# %Y-%m-%d %a-%H:%M
json.dump(
    {"instances":[instance]},
    open(json_path,"w")
)
os.environ["JSON_PATH"]  = json_path

In [5]:
instance

{'pickup_latitude': 0,
 'pickup_longitude': 0,
 'dropoff_latitude': 0,
 'dropoff_longitude': 0}

In [6]:
!gcloud ai-platform local predict --model-dir="${MODEL_PATH}" --json-request="${JSON_PATH}" --framework=tensorflow --signature-name="predict"

Instructions for updating:
non-resource variables are not supported in the long term
2021-06-17 15:24:28.816262: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-06-17 15:24:28.816630: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2021-06-17 15:24:28.816656: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-06-17 15:24:28.816694: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mirco-nani-dev-spd): /proc/driver/nvidia/version does not exist
2021-06-17 15:24:28.817687: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices n

# Setup

In [7]:
os.environ["AI_PLATFORM_PREDICTION_MODEL"]   = f"taxi{label}"
os.environ["AI_PLATFORM_PREDICTION_REGION"]  = "europe-west1"
os.environ["AI_PLATFORM_PREDICTION_VERSION"] = f"{flow_dir}_{model_name}"

In [8]:
!gcloud ai-platform models create ${AI_PLATFORM_PREDICTION_MODEL} --region=${AI_PLATFORM_PREDICTION_REGION}  --enable-logging

Using endpoint [https://europe-west1-ml.googleapis.com/]
Created ai platform model [projects/mlteam-ml-specialization-2021/models/taxitrip_seconds].


In [9]:
!gcloud ai-platform versions create ${AI_PLATFORM_PREDICTION_VERSION} \
--region=${AI_PLATFORM_PREDICTION_REGION} \
--model=${AI_PLATFORM_PREDICTION_MODEL} \
--origin=${MODEL_PATH} \
--runtime-version=2.4

Using endpoint [https://europe-west1-ml.googleapis.com/]
Creating version (this might take a few minutes)......done.                    


In [10]:
os.environ["AI_PLATFORM_PREDICTION_VERSION"]

'TaxiTS2017MinimalV01_20210521V02'

In [11]:
import json
import os


instance=dict(list(zip(numeric_columns,[0 for _ in numeric_columns]))+list(zip(categorical_columns, ["" for _ in categorical_columns])))
if "trip_start" in instance:
    instance["trip_start"] = "2017-03-03 Fry-09:00"
model_name=os.environ["AI_PLATFORM_PREDICTION_MODEL"]
region=os.environ["AI_PLATFORM_PREDICTION_REGION"]

os.environ["ENDPOINT"]=F"https://{region}-ml.googleapis.com/v1/projects/mlteam-ml-specialization-2021/models/{model_name}:predict"
# https://stackoverflow.com/questions/58134103/tf-how-do-i-setup-the-model-signature-correctly-for-serving-with-docker
os.environ["REQUEST_BODY"]=json.dumps({
    "instances": [instance],
    'signature_name': 'predict'
})

In [12]:
os.environ["REQUEST_BODY"]

'{"instances": [{"pickup_latitude": 0, "pickup_longitude": 0, "dropoff_latitude": 0, "dropoff_longitude": 0}], "signature_name": "predict"}'

In [13]:
!curl -d "${REQUEST_BODY}" -X POST ${ENDPOINT}?access_token\="$(gcloud auth application-default print-access-token)"

{
    "predictions": [[415.634583]
    ]
}

# teardown

In [14]:
! gcloud ai-platform versions delete ${AI_PLATFORM_PREDICTION_VERSION} --model=${AI_PLATFORM_PREDICTION_MODEL} --region=${AI_PLATFORM_PREDICTION_REGION} --quiet

Using endpoint [https://europe-west1-ml.googleapis.com/]
Deleting version [TaxiTS2017MinimalV01_20210521V02]......done.                 


In [15]:
! gcloud ai-platform models delete ${AI_PLATFORM_PREDICTION_MODEL} --region=${AI_PLATFORM_PREDICTION_REGION} --quiet

Using endpoint [https://europe-west1-ml.googleapis.com/]
Deleting model [taxitrip_seconds]...done.                                      
